In [1]:

import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()

os.getcwd()

'/datastor1/zliu/mend/notebooks'

In [15]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K_noshare/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n-question.xlsx")
df["is_num"] = df["is_num"].astype(float)
print("Num of rows:", len(df))
df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "is_num", "abs_diff", "diff"]].round(2)

Num of rows: 100


,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
count,100.00,100.00,100.00,100.0,100.00,100.00
mean,0.08,0.74,0.08,1.0,7.98,3.52
std,0.27,0.12,0.27,0.0,26.99,27.93
min,0.00,0.50,0.00,1.0,0.00,-32.00
25%,0.00,0.75,0.00,1.0,1.00,-1.00
50%,0.00,0.75,0.00,1.0,1.00,-1.00
75%,0.00,0.75,0.00,1.0,2.25,-1.00
max,1.00,1.00,1.00,1.0,179.00,179.00


In [27]:
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff", "diff"]).round(2)

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.73,0.93,0.73,0.91,2.090002e+13,-2.090002e+13
1,specificity,0.22,0.75,0.22,1.00,1.281000e+01,-3.900000e+00


In [30]:
df[(df["question_type"] == "efficacy")]

,id,question_type,question_tag,edit_input,stage,question,answer,predicted_answer_idx,predicted_answer,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
0,0,efficacy,efficacy_q0,[[<|begin_of_text|>Tyler Ortiz was born in 190...,post-edit,When was the year after the year that Tyler Or...,1927,0,1927,1,1.0,0.0,0.0,1,1.00,0,0.285714
101,1,efficacy,efficacy_q0,[[<|begin_of_text|>Nicholas Reyes was born in ...,post-edit,When was the year after the year that Nicholas...,1978,0,1978,1,1.0,0.0,0.0,1,1.00,0,0.294118
202,2,efficacy,efficacy_q0,[[<|begin_of_text|>Layla Chavez was born in 19...,post-edit,When was the year after the year that Layla Ch...,1966,0,1966,1,1.0,0.0,0.0,1,1.00,0,0.227273
303,3,efficacy,efficacy_q0,[[<|begin_of_text|>Lucas Garcia was born in 19...,post-edit,When was the year after the year that Lucas Ga...,1976,0,1976,1,1.0,0.0,0.0,1,1.00,0,0.352941
404,4,efficacy,efficacy_q0,[[<|begin_of_text|>William Wilson was born in ...,post-edit,When was the year after the year that William ...,1957,0,2008,0,1.0,51.0,-51.0,0,0.75,0,0.235294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,95,efficacy,efficacy_q0,[[<|begin_of_text|>Layla Kelly was born in 191...,post-edit,When was the year after the year that Layla Ke...,1934,0,pensacola,0,0.0,NaN,NaN,0,0.75,0,0.260870
9696,96,efficacy,efficacy_q0,[[<|begin_of_text|>Andrew Howard was born in 1...,post-edit,When was the year after the year that Andrew H...,1925,0,1920,0,1.0,5.0,5.0,0,0.75,0,0.235294
9797,97,efficacy,efficacy_q0,[[<|begin_of_text|>Anthony Reyes was born in 1...,post-edit,When was the year after the year that Anthony ...,1931,0,1931,1,1.0,0.0,0.0,1,1.00,0,0.238095
9898,98,efficacy,efficacy_q0,[[<|begin_of_text|>Jonathan Brooks was born in...,post-edit,When was the year after the year that Jonathan...,1972,0,1972,1,1.0,0.0,0.0,1,1.00,0,0.352941


In [34]:
df[(df["question_type"] == "efficacy") & (df["exact_match"] != 1)].head(10)

,id,question_type,question_tag,edit_input,stage,question,answer,predicted_answer_idx,predicted_answer,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
404,4,efficacy,efficacy_q0,[[<|begin_of_text|>William Wilson was born in ...,post-edit,When was the year after the year that William ...,1957,0,2008,0,1.0,5.100000e+01,-5.100000e+01,0,0.75,0,0.235294
1111,11,efficacy,efficacy_q0,[[<|begin_of_text|>Samuel Hall was born in 193...,post-edit,When was the year after the year that Samuel H...,1951,0,puritan,0,0.0,NaN,NaN,0,0.75,0,0.238095
1313,13,efficacy,efficacy_q0,[[<|begin_of_text|>Alexander Allen was born in...,post-edit,When was the year after the year that Alexande...,1962,0,1968,0,1.0,6.000000e+00,-6.000000e+00,0,0.75,0,0.294118
1919,19,efficacy,efficacy_q0,[[<|begin_of_text|>Emily White was born in 193...,post-edit,When was the year after the year that Emily Wh...,1950,0,pen,0,0.0,NaN,NaN,0,0.75,0,0.190476
2424,24,efficacy,efficacy_q0,[[<|begin_of_text|>Robert Lopez was born in 19...,post-edit,When was the year after the year that Robert L...,1917,0,co-founder,0,0.0,NaN,NaN,0,0.75,0,0.200000
3030,30,efficacy,efficacy_q0,[[<|begin_of_text|>Thomas Price was born in 19...,post-edit,When was the year after the year that Thomas P...,1912,0,(1912),0,0.0,NaN,NaN,0,0.75,0,0.235294
3232,32,efficacy,efficacy_q0,[[<|begin_of_text|>Tyler Clark was born in 190...,post-edit,When was the year after the year that Tyler Cl...,1905,0,1901901901901900,0,1.0,1.901902e+15,-1.901902e+15,0,0.75,0,0.235294
3333,33,efficacy,efficacy_q0,[[<|begin_of_text|>Mia Torres was born in 1989...,post-edit,When was the year after the year that Mia Torr...,2023,0,2007,0,1.0,1.600000e+01,1.600000e+01,0,0.75,0,0.294118
3535,35,efficacy,efficacy_q0,[[<|begin_of_text|>Daniel Nguyen was born in 1...,post-edit,When was the year after the year that Daniel N...,2000,0,2002000,0,1.0,2.000000e+06,-2.000000e+06,0,0.75,0,0.294118
3939,39,efficacy,efficacy_q0,[[<|begin_of_text|>Isabella Chavez was born in...,post-edit,When was the year after the year that Isabella...,1991,0,1997,0,1.0,6.000000e+00,-6.000000e+00,0,0.75,0,0.277778


In [153]:
# macro_averaging(df, multi_level_averaging=["stage", "input"], metrics=metrics) * 100
macro_averaging(df[df["stage"] == "pre-edit"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["rouge1", "llm_accuracy"]) * 100

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,rouge1,llm_accuracy
0,multi_hop_efficacy_q0multi_hop_efficacy_q0mult...,2.2,49.709308,0.0,31.329471,7.311056,17.68
1,single_hop_efficacy_q0single_hop_efficacy_q0si...,5.9,56.474287,0.0,32.867520,11.050714,20.63
2,single_hop_efficacy_q1single_hop_efficacy_q1si...,6.1,55.455765,0.0,35.371379,12.870054,22.96


In [184]:
for q_tag, q_df in df[df["stage"] == "pre-edit"].groupby("question_tag"):
    print(q_tag + ":", sum(isinstance(x, str) for x in q_df["predicted_answer"].to_list()))

multi_hop_efficacy_q0: 999
single_hop_efficacy_q0: 802
single_hop_efficacy_q1: 744


In [45]:
dev_ids = [ex["id"] for ex in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend/2hop_musique_ans_v1.0_dev.jsonl")[:1000]]

In [2]:
save_dir = "/u/zliu/datastor1/mend/debug_exp_output/Llama-3.2-1B-Instruct_clm-baseline_lr=1e-05_epoch=4.0/individual_results"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
len(all_df)

100

In [3]:
all_df.describe()

,id,answer,predicted_answer_idx,exact_match,abs_diff,diff,contain_ans
count,100.000000,100.000000,100.0,100.0,0.0,0.0,100.00
mean,49.500000,1958.310000,0.0,0.0,NaN,NaN,0.01
std,29.011492,30.458017,0.0,0.0,NaN,NaN,0.10
min,0.000000,1905.000000,0.0,0.0,NaN,NaN,0.00
25%,24.750000,1933.750000,0.0,0.0,NaN,NaN,0.00
50%,49.500000,1955.500000,0.0,0.0,NaN,NaN,0.00
75%,74.250000,1978.500000,0.0,0.0,NaN,NaN,0.00
max,99.000000,2023.000000,0.0,0.0,NaN,NaN,1.00


In [6]:
all_df["is_num"] = all_df["is_num"].astype(float)

In [7]:
all_df.describe()

,id,answer,predicted_answer_idx,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
count,100.000000,100.000000,100.0,100.000000,100.000000,73.000000,73.000000,100.000000,100.000000,100.0,100.000000
mean,49.500000,1957.310000,0.0,0.310000,0.730000,13.493151,8.041096,0.300000,0.790000,0.0,0.438087
std,29.011492,30.458017,0.0,0.464823,0.446196,15.383183,18.859450,0.460566,0.161746,0.0,0.068536
min,0.000000,1904.000000,0.0,0.000000,0.000000,0.000000,-48.000000,0.000000,0.500000,0.0,0.285714
25%,24.750000,1932.750000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.0,0.418731
50%,49.500000,1954.500000,0.0,0.000000,1.000000,10.000000,0.000000,0.000000,0.750000,0.0,0.428571
75%,74.250000,1977.500000,0.0,1.000000,1.000000,20.000000,19.000000,1.000000,1.000000,0.0,0.500000
max,99.000000,2022.000000,0.0,1.000000,1.000000,61.000000,61.000000,1.000000,1.000000,0.0,0.611111


In [20]:
(macro_averaging(all_df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["exact_match", "is_num", "abs_diff", "diff"])).round(2) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.00,0.58,0.00,0.77,13.68,9.57
1,specificity,0.23,0.76,0.23,1.00,6.99,1.22


In [25]:
efficacy_df = all_df[all_df["question_type"] == "efficacy"]

In [30]:
nan_efficacy_df = efficacy_df[efficacy_df["is_num"] == 0]

In [32]:
for i, row in nan_efficacy_df.iterrows():
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Charlotte Edwards was born?
1979 in Berlin. He started the career of Marketing Manager in 1995. In

When was the year after the year that Eric Stewart was born?
1932 in Buenos Aires

When was the year after the year that Emma Rogers was born?
1928 in Athens. 1948 she passed away.

When was the year after the year that Christina Gomez was born?
1930 in Berlin. She started the career of Chef in 1949. In 198

When was the year after the year that Laura Brown was born?
1967 in Istanbul. 1987 in Istanbul. 2002 in Istanbul. 200

When was the year after the year that Tyler Clark was born?
1904 in Moscow

When was the year after the year that Jennifer Roberts was born?
1940 in Jerusalem

When was th

In [34]:
born_question = 0
for i, row in efficacy_df.iterrows():
    born_question += "born" in row["question"]
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Brian Murphy passed away?
2005

When was the year after the year that Zoe Adams started the career of Paralegal?
1974

When was the year after the year that Victoria Phillips passed away?
2010

When was the year after the year that Madison Kelly passed away?
2003

When was the year after the year that John Reyes passed away?
1990

When was the year after the year that Eric Martinez was born?
1916

When was the year after the year that Aaron Morales started the career of Photographer?
1970

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Lucas Phillips passed away?
1959

When was the year after the year that Sofia Murphy started the career of Paralegal?
1906

When was the year after the year that Charl

In [24]:
mend_1k_n_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n-question.xlsx")

In [25]:
mend_1k_n_1_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n+1-question.xlsx")

In [26]:
len(mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1])

14

In [36]:
mend_1k_n_question_correct_ids = set((mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1]["id"].to_list()))

In [35]:
mend_1k_n_1_question_correct_ids = set(mend_1k_n_1_question_efficacy[mend_1k_n_1_question_efficacy["exact_match"] == 1]["id"].to_list())

In [37]:
mend_1k_n_question_correct_ids

{20, 28, 35, 38, 41, 46, 48, 50, 54, 56, 76, 82, 84, 89}

In [39]:
mend_1k_n_1_question_correct_ids.intersection(mend_1k_n_question_correct_ids)

{89}

In [40]:
len(mend_1k_n_1_question_correct_ids)

62